Aunque el lenguaje de este notebook pueda parecer coloquial, su objetivo no es otro que hacer una aproximación de lo que el autoencoder hace para entenderlo mejor.En esta versión del autoencoder, explico como veo el funcionamiento de esta red neuronal e intento desglosar todo lo que incluye la versión oficial que nos pasó Carlos. En definitiva el autoencoder está constituido por un encoder más un decoder como ya sabiamos. Nosotros prestamos atención al espacio que tenemos entre ambos, creado un vector lantente para posteriormente volver a reproducir la salida mediante el autoencoder. Se generan dos vectores que hace de media y otro de desviación típica. Ambos se usan para producir un nuevo vector asegurandonos que su distribución cumple con una distribución Gaussiana. La generación se hace a partir de un vector random que generamos cumpliendo una distribución normal N(0,1). Las funciones de pérdida son: una típica normal que calcula las diferencias entre output e input y otra de pérdida (Kullback-Leibler (KL)) que lo que hace es regularizar los pesos para asegurarnos que se cumple la distribución gaussiana, se intenta minimizar éstas perdidas indudablemente. 

#### Datos
Tenemos características genómicas de mil tumores tomadas de 33 tipos diferentes de cáncer.
- **5000 Variables(features)->genes expresados-protein coding.**
- **10459 Observaciones -> gene expression para 10459 tumores (Escogidos entre los más irregulares y variables a través de la MAD).**

#### Entrenamiento
- Optimizador -> adam
- Encoding->rectified linear units and batch normalization.
- Decoding->sigmoid activation.


### Importaciones

Como importaciones tenemos las herramientas que necesitamos para crear las redes neuronales y llevar a cabo la experimentación.

In [26]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf

from keras import backend as K
from keras.layers import (Input, InputLayer, Dense, Lambda, Layer, Add, Multiply)
from keras.models import Model, Sequential
import keras

import pandas as pd
from matplotlib.ticker import FormatStrFormatter
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.layers.normalization import BatchNormalization
from IPython.display import SVG

### Versiones

In [3]:
print(keras.__version__)
tf.__version__

2.2.4


'1.13.0-rc2'

### Cargamos los datos

Como hemos comentado tenemos observaciones con los genes mayormente expresados(5000) de 10459 tumores.

In [7]:
#rnaseq_file = os.path.join('data', 'pancan_scaled_zeroone_rnaseq.tsv')
rnaseq_df = pd.read_csv('pancan_scaled_zeroone_rnaseq.tsv.gz',sep='\t', index_col=0)

In [6]:
print(rnaseq_df.shape)
rnaseq_df.head(5)

(10459, 5000)


RPS4Y1      XIST      KRT5  AGR2   CEACAM5     KRT6A  \
TCGA-02-0047-01  0.678296  0.289910  0.034230   0.0  0.000000  0.084731   
TCGA-02-0055-01  0.200633  0.654917  0.181993   0.0  0.000000  0.100606   
TCGA-02-2483-01  0.785980  0.140842  0.081082   0.0  0.000000  0.000000   
TCGA-02-2485-01  0.720258  0.122554  0.180042   0.0  0.000000  0.000000   
TCGA-02-2486-01  0.767127  0.210393  0.034017   0.0  0.061161  0.000000   

                    KRT14   CEACAM6     DDX3Y     KDM5D    ...      FAM129A  \
TCGA-02-0047-01  0.031863  0.037709  0.746797  0.687833    ...     0.440610   
TCGA-02-0055-01  0.050011  0.092586  0.103725  0.140642    ...     0.620658   
TCGA-02-2483-01  0.000000  0.000000  0.730648  0.657189    ...     0.437658   
TCGA-02-2485-01  0.000000  0.000000  0.720306  0.719855    ...     0.553306   
TCGA-02-2486-01  0.053021  0.000000  0.739546  0.665684    ...     0.601268   

                  C8orf48    CDK5R1    FAM81A  C13orf18     GDPD3     SMAGP  \
TCGA-02-0047-01  0.428782  0.732819  0.634340  0.580662  0.294313  0.458134   
TCGA-02-0055-01  0.363207  0.592269  0.602755  0.610192  0.374569  0.722420   
TCGA-02-2483-01  0.471489  0.868774  0.471141  0.487212  0.385521  0.466642   
TCGA-02-2485-01  0.373344  0.818608  0.691962  0.635023  0.430647  0.453690   
TCGA-02-2486-01  0.379943  0.506839  0.684320  0.607821  0.320113  0.476190   

                  C2orf85   POU5F1B     CHST2  
TCGA-02-0047-01  0.478219  0.168263  0.638497  
TCGA-02-0055-01  0.271356  0.160465  0.602560  
TCGA-02-2483-01  0.784059  0.160797  0.557074  
TCGA-02-2485-01  0.364494  0.161363  0.607895  
TCGA-02-2486-01  0.122722  0.389544  0.698548  

[5 rows x 5000 columns]

Separamos en train y en test

In [8]:
test_set_percent = 0.15
rnaseq_test_df = rnaseq_df.sample(frac=test_set_percent)
rnaseq_train_df = rnaseq_df.drop(rnaseq_test_df.index)

In [10]:
gene_expression=rnaseq_df.columns.tolist()
print(gene_expression)

['RPS4Y1', 'XIST', 'KRT5', 'AGR2', 'CEACAM5', 'KRT6A', 'KRT14', 'CEACAM6', 'DDX3Y', 'KDM5D', 'SLC34A2', 'TMPRSS4', 'KRT6B', 'GPX2', 'SERPINB5', 'HMGCS2', 'PIGR', 'KRT16', 'KRT17', 'UGT1A6', 'FOXA1', 'FXYD3', 'HNF1B', 'KRT19', 'RAB25', 'DSG3', 'USP9Y', 'EIF1AY', 'LCN2', 'KRT15', 'PRAME', 'KRT13', 'CALML3', 'KLK11', 'AZGP1', 'GRHL2', 'KRT7', 'AKR1B10', 'DSC3', 'PTPRZ1', 'CXCL17', 'LRP2', 'UTY', 'AGR3', 'USH1C', 'HNF4A', 'GJB1', 'SPDEF', 'LOC642587', 'S100P', 'TNS4', 'SFTPB', 'GSTA1', 'MUC5B', 'KIF1A', 'KCNJ16', 'TFF1', 'MSLN', 'SLC44A4', 'TRIM29', 'PI3', 'GSTM1', 'TSPAN8', 'SOX2', 'S100A14', 'COL17A1', 'PKP1', 'ANXA8', 'TACSTD2', 'KRT6C', 'KLK10', 'CLDN3', 'SPINK1', 'PPP1R1B', 'ALDH3B2', 'CTSE', 'HGD', 'LY6D', 'CA9', 'PLA2G2A', 'HPN', 'OLFM4', 'CDHR5', 'SCEL', 'MUC13', 'ZFY', 'FAM83A', 'GDA', 'EHF', 'WDR72', 'CLCA2', 'SFN', 'ITGB6', 'CYorf15A', 'KRT23', 'SLC6A14', 'KLK6', 'VIL1', 'ESRP1', 'CLDN2', 'TMPRSS2', 'PCK1', 'SPRR1B', 'HOXB13', 'HOXC10', 'FGFBP1', 'DDC', 'FGG', 'UGT1A9', 'PKP3', 

### Variables e hiperparámetros

In [11]:
original_dim=rnaseq_df.shape[1]
latent_dim = 100

batch_size=50
epochs = 50
learning_rate=0.0005

beta = K.variable(0)
kappa = 1
epsilon_std = 1.0

Instructions for updating:
Colocations handled automatically by placer.


### Encoder

Incluyendo **batch_normalization** han encontrado que se entrena más rápido y además con una activación de caraterísticas más heterogenea, batch_normalization añade una regulación de características escalando la activación a media=0 y varianza=1.Es como un método para normalizar los inputs de cada capa. 

In [16]:
##Creamos la capa de entrada
inputs = Input(shape=(original_dim,),name='encoder_input')
##Creamos capa intermedia
x_encoder = Dense(latent_dim, activation='relu',kernel_initializer='glorot_uniform')(inputs)
##Creamos dos capas de salida,media y log_desviacion(se usa en lugar de la estándar)
z_mean_encoded = BatchNormalization()(x_encoder)
log_desviacion = BatchNormalization()(x_encoder)

### Decoder

In [17]:
##Creamos capa de entrada del decodificador
latent_vector = Input(shape=(latent_dim,), name='latent_vector')
outputs = Dense(original_dim,kernel_initializer='glorot_uniform',activation='sigmoid')(latent_vector)

### Latent vector

In [20]:
def sampler(args):
    z_mean, z_log = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # mean and log of variance of Q(z|X)
    # we sample from the standard normal a matrix of batch_size * latent_size (taking into account minibatches)
    epsilon = K.random_normal(shape=(batch,dim))# by default, random_normal has mean=0 and std=1.0
    # sampling from Z~N(μ, σ^2) is the same as sampling from μ + σX, X~N(0,1)
    return z_mean + K.exp(0.5 * z_log) * epsilon
  
z = Lambda(sampler)([z_mean_encoded, log_desviacion])

### Compilación de modelos-VAE

In [22]:
##Instanciamos el encoder
encoder = Model(inputs,[z_mean_encoded, log_desviacion,z],name='encoder')
encoder.summary()
##Instanciamos el decoder
decoder = Model(latent_vector,outputs,name='decoder')
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 5000)         0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 100)          500100      encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 100)          400         dense_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_5 (BatchNor (None, 100)          400         dense_4[0][0]                    
____________________________________________________________________________________________

En este ejemplo no incluimos:

El **CustomVariationalLayer()** incluye la función de pérdida VAE (reconstruction + (beta * KL)), que es lo que impulsará a nuestro modelo a aprender una representación interpretable del espacio de expresión génica.

El VAE compilado con un optimizador Adam y una función de pérdida personalizada incorporada. El parámetro loss_weights garantiza que la versión beta se actualice en cada devolución de llamada de final de época

In [39]:
##Creamos el modelo final
outputs_final= decoder(encoder(inputs)[2])
vae = Model(inputs,outputs_final,name='vae')
vae.compile(optimizer='adam', loss=vae_loss, loss_weights=[beta])
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 5000)              0         
_________________________________________________________________
encoder (Model)              [(None, 100), (None, 100) 500900    
_________________________________________________________________
decoder (Model)              (None, 5000)              505000    
Total params: 1,005,900
Trainable params: 1,005,500
Non-trainable params: 400
_________________________________________________________________


### VAE loss

In [40]:
def vae_loss(inputs, outputs):
    reconstruction_loss = K.sum(K.square(outputs-inputs))
    kl_loss = - 0.5 * K.mean(1 + log_desviacion - K.square(z_mean_encoded) - K.exp(log_desviacion), axis=-1)
    total_loss = K.mean(reconstruction_loss + kl_loss)    
    return total_loss

### Entrenamiento

In [41]:
vae.fit(rnaseq_train_df,
        rnaseq_train_df,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(rnaseq_test_df,rnaseq_test_df))


Instructions for updating:
Use tf.cast instead.
Train on 8890 samples, validate on 1569 samples
Epoch 1/50
8890/8890 [==============================] - 12s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/50
8890/8890 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/50
8890/8890 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/50
8890/8890 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/50
8890/8890 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/50
8890/8890 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/50
8890/8890 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/50
8890/8890 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/50
8890/8890 [=

### Reconstrucción

In [57]:
# How well does the model reconstruct the input RNAseq data
encoder = Model(inputs, z_mean_encoded)
encoded_rnaseq_df = encoder.predict_on_batch(rnaseq_df)
input_rnaseq_reconstruct = decoder.predict(np.array(encoded_rnaseq_df))
input_rnaseq_reconstruct = pd.DataFrame(input_rnaseq_reconstruct, index=rnaseq_df.index,
                                        columns=rnaseq_df.columns)
input_rnaseq_reconstruct.head(2)

RPS4Y1      XIST      KRT5      AGR2   CEACAM5     KRT6A  \
TCGA-02-0047-01  0.494333  0.415199  0.517435  0.505923  0.415287  0.554540   
TCGA-02-0055-01  0.473873  0.447998  0.535658  0.589215  0.441431  0.504417   

                    KRT14   CEACAM6     DDX3Y     KDM5D    ...      FAM129A  \
TCGA-02-0047-01  0.464486  0.502964  0.603659  0.558587    ...     0.487442   
TCGA-02-0055-01  0.531021  0.537004  0.545304  0.563314    ...     0.513838   

                  C8orf48    CDK5R1    FAM81A  C13orf18     GDPD3     SMAGP  \
TCGA-02-0047-01  0.551457  0.527629  0.475395  0.601340  0.531287  0.474413   
TCGA-02-0055-01  0.503204  0.479907  0.500441  0.505613  0.521935  0.488860   

                  C2orf85   POU5F1B     CHST2  
TCGA-02-0047-01  0.471939  0.473112  0.549920  
TCGA-02-0055-01  0.532566  0.502805  0.530295  

[2 rows x 5000 columns]